#### 三种启发式算法的示例代码

我们将用这些算法尝试找到函数 f(x)=x 


  在区间 [−5,5] 内的最小值（理论最小值是 x=0 时 f(x)=0）。


a) 遗传算法 (Genetic Algorithm, GA)

遗传算法模拟生物进化过程，通过选择、交叉和变异来迭代改进解。



In [1]:
import random
import numpy as np

# 目标函数 (越小越好)
def objective_function(x):
  return x**2

# --- GA 参数 ---
POPULATION_SIZE = 50     # 种群大小
GENOME_LENGTH = 1       # 基因长度 (我们这里解是1维的)
MUTATION_RATE = 0.1     # 变异率
CROSSOVER_RATE = 0.8    # 交叉率
NUM_GENERATIONS = 50    # 迭代次数
BOUNDS = [-5, 5]        # 解的范围

# --- GA 实现 ---

# 初始化种群 (这里用浮点数表示基因)
population = np.random.uniform(BOUNDS[0], BOUNDS[1], (POPULATION_SIZE, GENOME_LENGTH))

# 迭代进化
for generation in range(NUM_GENERATIONS):
  # 1. 计算适应度 (适应度越高越好，所以用 1 / (1 + f(x)) 或 max(f) - f(x))
  #    为简单起见，直接用目标函数值排序，选小的
  fitness = np.array([objective_function(ind[0]) for ind in population])
  sorted_indices = np.argsort(fitness)
  population = population[sorted_indices] # 按适应度排序
  fitness = fitness[sorted_indices]

  # 保留最佳个体 (精英主义)
  new_population = [population[0].copy()] # 复制最佳个体

  # 2. 选择、交叉、变异产生新个体
  while len(new_population) < POPULATION_SIZE:
    # 简单的锦标赛选择 (选两个，取适应度好的)
    idx1, idx2 = np.random.choice(range(POPULATION_SIZE), 2, replace=False)
    parent1 = population[idx1] if fitness[idx1] < fitness[idx2] else population[idx2]

    idx1, idx2 = np.random.choice(range(POPULATION_SIZE), 2, replace=False)
    parent2 = population[idx1] if fitness[idx1] < fitness[idx2] else population[idx2]

    # 3. 交叉 (算术交叉)
    child = parent1.copy()
    if random.random() < CROSSOVER_RATE:
      alpha = random.random()
      child = alpha * parent1 + (1 - alpha) * parent2

    # 4. 变异 (高斯变异)
    if random.random() < MUTATION_RATE:
      child += np.random.normal(0, 0.5) # 增加一点随机扰动

    # 边界处理
    child = np.clip(child, BOUNDS[0], BOUNDS[1])

    new_population.append(child)

  population = np.array(new_population)

# 迭代结束，找到最佳解
best_solution = population[0]
best_fitness = objective_function(best_solution[0])

print(f"遗传算法:")
print(f"  找到的最佳解 x = {best_solution[0]:.4f}")
print(f"  对应的目标函数值 f(x) = {best_fitness:.4f}")

遗传算法:
  找到的最佳解 x = 0.0000
  对应的目标函数值 f(x) = 0.0000


b) 粒子群算法 (Particle Swarm Optimization, PSO)


PSO 模拟鸟群觅食行为，每个粒子根据自身经验和群体经验更新速度和位置。

In [2]:
import random
import numpy as np

# 目标函数
def objective_function(x):
  return x**2

# --- PSO 参数 ---
NUM_PARTICLES = 30      # 粒子数量
DIMENSIONS = 1         # 解的维度
NUM_ITERATIONS = 50     # 迭代次数
BOUNDS = [-5, 5]        # 搜索范围
W = 0.5                # 惯性权重
C1 = 1.5               # 个体学习因子
C2 = 1.5               # 社会学习因子

# --- PSO 实现 ---

# 初始化粒子位置和速度
particles_pos = np.random.uniform(BOUNDS[0], BOUNDS[1], (NUM_PARTICLES, DIMENSIONS))
particles_vel = np.random.uniform(-1, 1, (NUM_PARTICLES, DIMENSIONS))

# 初始化个体最佳位置 (pbest) 和全局最佳位置 (gbest)
pbest_pos = particles_pos.copy()
pbest_fitness = np.array([objective_function(p[0]) for p in particles_pos])

gbest_idx = np.argmin(pbest_fitness)
gbest_pos = pbest_pos[gbest_idx].copy()
gbest_fitness = pbest_fitness[gbest_idx]

# 迭代优化
for _ in range(NUM_ITERATIONS):
  for i in range(NUM_PARTICLES):
    # 评估当前粒子适应度
    current_fitness = objective_function(particles_pos[i, 0])

    # 更新个体最佳 pbest
    if current_fitness < pbest_fitness[i]:
      pbest_fitness[i] = current_fitness
      pbest_pos[i] = particles_pos[i].copy()

      # 更新全局最佳 gbest
      if current_fitness < gbest_fitness:
        gbest_fitness = current_fitness
        gbest_pos = particles_pos[i].copy()

    # 更新速度
    r1, r2 = random.random(), random.random()
    cognitive_vel = C1 * r1 * (pbest_pos[i] - particles_pos[i])
    social_vel = C2 * r2 * (gbest_pos - particles_pos[i])
    particles_vel[i] = W * particles_vel[i] + cognitive_vel + social_vel

    # 更新位置
    particles_pos[i] += particles_vel[i]

    # 边界处理
    particles_pos[i] = np.clip(particles_pos[i], BOUNDS[0], BOUNDS[1])

print(f"粒子群算法:")
print(f"  找到的最佳解 x = {gbest_pos[0]:.4f}")
print(f"  对应的目标函数值 f(x) = {gbest_fitness:.4f}")

粒子群算法:
  找到的最佳解 x = -0.0000
  对应的目标函数值 f(x) = 0.0000


c) 模拟退火算法 (Simulated Annealing, SA)

SA 模拟固体材料退火过程，允许在高温时接受较差的解以跳出局部最优，随温度降低逐渐稳定到最优解。

In [3]:
import random
import numpy as np
import math

# 目标函数
def objective_function(x):
  return x**2

# --- SA 参数 ---
INITIAL_TEMP = 100.0    # 初始温度
FINAL_TEMP = 0.1       # 最终温度
COOLING_RATE = 0.95    # 冷却率 (alpha)
BOUNDS = [-5, 5]        # 搜索范围
STEPS_PER_TEMP = 50     # 每个温度下的迭代次数

# --- SA 实现 ---

# 初始化当前解
current_solution = random.uniform(BOUNDS[0], BOUNDS[1])
current_fitness = objective_function(current_solution)

best_solution = current_solution
best_fitness = current_fitness

temperature = INITIAL_TEMP

# 退火过程
while temperature > FINAL_TEMP:
  for _ in range(STEPS_PER_TEMP):
    # 产生邻近解 (小扰动)
    neighbor_solution = current_solution + np.random.normal(0, 0.5) # 高斯扰动
    neighbor_solution = np.clip(neighbor_solution, BOUNDS[0], BOUNDS[1]) # 边界处理
    neighbor_fitness = objective_function(neighbor_solution)

    # 计算能量差 (目标函数值差)
    delta_e = neighbor_fitness - current_fitness

    # 判断是否接受新解
    if delta_e < 0: # 新解更好，直接接受
      current_solution = neighbor_solution
      current_fitness = neighbor_fitness
      # 更新全局最优
      if current_fitness < best_fitness:
        best_solution = current_solution
        best_fitness = current_fitness
    else: # 新解更差，按概率接受 (Metropolis准则)
      acceptance_prob = math.exp(-delta_e / temperature)
      if random.random() < acceptance_prob:
        current_solution = neighbor_solution
        current_fitness = neighbor_fitness

  # 降温
  temperature *= COOLING_RATE

print(f"模拟退火算法:")
print(f"  找到的最佳解 x = {best_solution:.4f}")
print(f"  对应的目标函数值 f(x) = {best_fitness:.4f}")

模拟退火算法:
  找到的最佳解 x = 0.0002
  对应的目标函数值 f(x) = 0.0000
